In [1]:
import tweepy
from datetime import datetime,timezone
import pytz
import pandas as pd
import time

In [2]:
# Twitter認証情報
consumer_key        = 'hophVgLGWBfTJLaqD9E0pSyO1'
consumer_secret     = '67gkgbvLlfIIr8j1nSW1VCss1OVUK66vs00dAXqZ7VWYUTKGBX'
access_token        = '1611281223095300102-KrgKZvwMVrjh4QuX2q1uDKwHeJqUzl'
access_token_secret = '9dbM9QTVa1wxAZnmOtwLF1u84wSQVbrWlDoA3G66TS48s'

#日本のWOEID
woeid = 23424856

#何件のツイートを取得するか
item_number = 100

In [3]:
#関数:　UTCをJSTに変換する
def change_time_JST(u_time):
    #イギリスのtimezoneを設定するために再定義する
    utc_time = datetime(u_time.year, u_time.month,u_time.day, \
    u_time.hour,u_time.minute,u_time.second, tzinfo=timezone.utc)
    #タイムゾーンを日本時刻に変換
    jst_time = utc_time.astimezone(pytz.timezone("Asia/Tokyo"))
    # 文字列で返す
    str_time = jst_time.strftime("%Y-%m-%d_%H:%M:%S")
    return str_time

In [4]:
#Twitterの認証
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
# スクレイピング実施
# 50個のトレンドを取得し、その検索結果で得られたツイートを100件取得する（それを3セット繰り返す）
# トレンドについては、1時間に1回更新する

tw_data = []
today_date_pre = datetime.now().day
save_path = './result/tweet_data_' + datetime.now().strftime('%Y%m%d') + '.csv'

while True:
    # 時間情報の取得（0時〜7時の間の情報は取得しない）
    now_hour = datetime.now().hour
    if now_hour < 7:
        time.sleep(3600)
        continue
        
    # 今日の日付を取得
    today_date = datetime.now().day
    # 1日にごとに保存するファイルを変更する
    if today_date_pre != today_date:
        tw_data = []
        save_path = './result/tweet_data_' + datetime.now().strftime('%Y%m%d') + '.csv'
    today_date_pre = today_date
    
    #トレンド一覧取得
    trends = api.get_place_trends(woeid)
    trends_list = [trend['name'] for trend in trends[0]['trends']]
    trends_interval = 1800
    start_time = time.time()
    # 50種類のトレンドそれぞれに対して、item_number件づつ3セットツイートの取得を行う。重複防止のため1セット終わるごとに10分感覚をあける
    for i in range(3):
        for trend in trends_list:
            search_word = trend
            #検索条件を元にツイートを抽出
            time.sleep(5)
            try:
                tweets = tweepy.Cursor(api.search_tweets,q=search_word, tweet_mode='extended',result_type="mixed",lang='ja').items(item_number)
            except TooManyRequests:
                print("detect TooManyRequests")
                time.sleep(600)
                tweets = tweepy.Cursor(api.search_tweets,q=search_word, tweet_mode='extended',result_type="mixed",lang='ja').items(item_number)
            '''
            引数説明
            result type
                "recent"： 時系列で最新のツイートを検索
                "popular"： 人気のあるツイートを検索
                "mixed"： 上記を混ぜたもの。
            
            tweeet_mode
                "extended" : tweet文を省略せずに取得
            '''
            for tweet in tweets:
                '''
                # フォロワーが 100 ~ 5000人以下のツイートは取得させない。
                num_follower = tweet.user.followers_count
                if num_follower < 100 or 5000 < num_follower:
                    continue
                '''
                #ツイート時刻とユーザのアカウント作成時刻を日本時刻にする
                tweet_time = change_time_JST(tweet.created_at)
                create_account_time = change_time_JST(tweet.user.created_at)
                #tweet_dataの配列に取得したい情報を入れていく
                tw_data.append([
                    tweet.id,
                    tweet_time,
                    tweet.full_text,
                    tweet.favorite_count, 
                    tweet.retweet_count, 
                    tweet.user.id, 
                    tweet.user.screen_name,
                    tweet.user.name,
                    tweet.user.description,
                    tweet.user.friends_count,
                    tweet.user.followers_count,
                    create_account_time,
                    tweet.user.following,
                    tweet.user.profile_image_url,
                    tweet.user.profile_background_image_url,
                    tweet.user.url
                                   ])

            #取り出したデータをpandasのDataFrameに変換
            #CSVファイルに出力するときの列の名前を定義
            labels=[
                'ツイートID',
                'ツイート時刻',
                'ツイート本文',
                'いいね数',
                'リツイート数',
                'ID',
                'ユーザー名',
                'アカウント名',
                '自己紹介文',
                'フォロー数',
                'フォロワー数',
                'アカウント作成日時',
                '自分のフォロー状況',
                'アイコン画像URL',
                'ヘッダー画像URL',
                'WEBサイト'
                ]
            #tw_dataのリストをpandasのDataFrameに変換
            df = pd.DataFrame(tw_data,columns=labels)

            
            #CSVファイルを出力する
            df.to_csv(save_path, encoding='utf-8-sig',index=False)
        time.sleep(600)
    if trends_interval > time.time() - start_time:
        time.sleep(trends_interval - (time.time() - start_time))